## Import

In [14]:
%matplotlib inline
import os, pickle
import numpy as np
import open3d as o3d
import geom3d, fit3d
import matplotlib.pyplot as plt
from myShape3d import Plane, Sphere, Cone
from cylinder_fitting import fit


from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
from feature_extractor import surflet_pairs_feature, get_histogram

PCA_THRESHOLD = 0.015
PLY_PATH = './dataset/ply/test/pointCloud'
SAVE_CLS_PATH = './res_dict/test_label_runB.pkl'
SAVE_PARAM_PATH = './dataset/ply/test/GTpointCloud'
LEN_TESTSET = len(next(os.walk(PLY_PATH))[2])

## Classify

### Utils

In [15]:
def knn_predict(model, xs, labels, honv=True):
    if not honv: 
        return model.predict(xs)
    counts = [len(x) for x in xs]
    data = np.concatenate(xs, axis=0)
    probs = model.predict_proba(data)

    pred = []
    head = 0
    for count in counts:
        p = probs[head:head+count, :].sum(axis=0)
        pred.append(labels[p.argmax()])
        head += count
    return pred

def knn_inference(model, labels, honv=True, path=PLY_PATH):
    test_label = {}
    if not honv:
        xs = []
        for id in tqdm(range(1, LEN_TESTSET + 1)):
            xs.append(surflet_pairs_feature(filepath=os.path.join(path, f"pointCloud{id}.ply")))
        xs = np.array(xs)
        ys = knn_predict(model, xs, labels=labels, honv=False)
    else:
        xs = []
        for id in tqdm(range(1, LEN_TESTSET + 1)):
            hists = get_histogram(filepath=os.path.join(path, f"pointCloud{id}.ply"), bins=25, max_ratio=0.8)
            xs.append(np.array(hists))
        ys = knn_predict(model, xs, labels=labels, honv=True)

    for id, y in enumerate(ys, 1):
        test_label[id] = y
    return test_label


### Plane vs others

In [16]:
test_label = {}
knn_12 = pickle.load(open('./honv/is_plane_25_08_03_full/model.pkl', 'rb'))
test_label = knn_inference(knn_12, [1, 2], honv=True)
del knn_12

  0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
print('[+] Remove plane false-accepts:')
false_accepts = 0
for id in range(1, LEN_TESTSET + 1):
    if test_label[id] == 1:
        x = np.asarray(o3d.io.read_point_cloud(os.path.join(PLY_PATH, f"pointCloud{id}.ply")).points)
        model = PCA()
        model.fit_transform(x)
        # print(min(model.explained_variance_ratio_))
        if min(model.explained_variance_ratio_) > PCA_THRESHOLD:
            test_label[id] = 2
            false_accepts += 1
print(f"\t{false_accepts} plane false-accepts")

[+] Remove plane false-accepts:
	1 plane false-accepts


### Cylinder-Cone vs. Sphere-Torus

In [18]:
knn_23 = pickle.load(open('./honv/2_class_25_08_05_full/model.pkl', 'rb'))
for id, label in knn_inference(knn_23, [2, 3], honv=True).items():
    if test_label[id] == 2:
        test_label[id] = label
del knn_23

  0%|          | 0/15 [00:00<?, ?it/s]

### Sphere vs. Torus

In [19]:
knn_35 = pickle.load(open('./SP/35_15_full/model.pkl', 'rb'))
for id, label in knn_inference(knn_35, [3, 5], honv=False).items():
    if test_label[id] == 3:
        test_label[id] = label
del knn_35

  0%|          | 0/15 [00:00<?, ?it/s]

### Cylinder vs. Cone

In [20]:
knn_24 = pickle.load(open('./honv/cyl-cone_25_08_05_full/model.pkl', 'rb'))
for id, label in knn_inference(knn_24, [2, 4], honv=True).items():
    if test_label[id] == 2:
        test_label[id] = label
del knn_24

  0%|          | 0/15 [00:00<?, ?it/s]

### Save results

In [21]:
pickle.dump(test_label, open(SAVE_CLS_PATH, 'wb'))

### Visualize results

In [22]:
name = ['', 'plane', 'cylinder', 'sphere', 'cone', 'torus']
LEN_DEMO = 15
for i in range(1, LEN_DEMO + 1):
    pts = np.asarray(o3d.io.read_point_cloud(os.path.join(PLY_PATH, f"pointCloud{i}.ply")).points)
    print("[+]", name[test_label[i]])

[+] sphere
[+] cylinder
[+] torus
[+] sphere
[+] sphere
[+] cone
[+] cylinder
[+] sphere
[+] sphere
[+] sphere
[+] cone
[+] plane
[+] sphere
[+] cylinder
[+] cone


## Fit shape

### Load label

In [26]:
test_label = pickle.load(open(SAVE_CLS_PATH, 'rb'))

### Utils

In [27]:
def generate_shape_params(pc, param_file, shape_type=None):
    if shape_type=='plane':
        generate_plane_result(pc, param_file)
    elif shape_type=='cylinder':
        generate_cylinder_result(pc, param_file)
    elif shape_type=='sphere':
        generate_sphere_result(pc, param_file)
    elif shape_type=='cone':
        generate_cone_result(pc, param_file)
    elif shape_type=='torus':
        generate_torus_result(pc, param_file)
    else:
        shape_type = None

    assert shape_type != None, "Invalid shape type"

def generate_plane_result(pc, param_file):
    plane = Plane()
    normal, center, loss, inliers = plane.fit(pc)

    param_file.write('1\n')
    for i in normal:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')
    for i in center:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')

    param_file.close()

def generate_cylinder_result(pc, param_file):
    pts = np.asarray(pc.points)
    axis, center, radius, fit_err = fit(pts)
            
    param_file.write('2\n')
    param_file.write(str(np.format_float_positional(radius, precision=9, trim='-')) + '\n')
    for i in axis:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')
    for i in center:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')

    param_file.close()

def generate_sphere_result(pc, param_file):
    sphere = Sphere()
    center, radius, inliers = sphere.fit(pc)

    param_file.write('3\n')
    param_file.write(str(np.format_float_positional(radius, precision=9, trim='-')) + '\n')
    for i in center:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')

    param_file.close()
    
def generate_cone_result(pc, param_file):
    cone = Cone()
    apex, axis, theta, mean_dist = cone.fit(pc, maxIteration=10000)

    param_file.write('4\n')
    param_file.write(str(np.format_float_positional(theta, precision=9, trim='-')) + '\n')
    for i in axis:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')
    for i in apex:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')

    param_file.close()

    
def generate_torus_result(pc, param_file):
    pts = np.asarray(pc.points)
    # c_init = np.mean(pts, axis=0)
    # initial_guess = geom3d.Torus([c_init[0], c_init[1], c_init[2]], [0, 0, 1], 1, 0.1)
    initial_guess = geom3d.Torus([0, 0, 0], [0, 0, 1], 1, 0.1)
    torus = fit3d.torus_fit(pts, initial_guess=initial_guess)
    axis = np.array(torus.direction)
    axis = axis / np.linalg.norm(axis)
    center = np.array(torus.center)
    majorRadius = torus.major_radius
    minorRadius = torus.minor_radius

    param_file.write('5\n')
    param_file.write(str(np.format_float_positional(majorRadius, precision=9, trim='-')) + '\n')
    param_file.write(str(np.format_float_positional(minorRadius, precision=9, trim='-')) + '\n')
    for i in axis:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')
    for i in center:
        param_file.write(str(np.format_float_positional(i, precision=9, trim='-')))
        param_file.write('\n')

    param_file.close()

In [28]:
name = ['', 'plane', 'cylinder', 'sphere', 'cone', 'torus']

for id in tqdm(range(1, LEN_TESTSET + 1)):
    label = test_label[id]
    pc= o3d.io.read_point_cloud(os.path.join(PLY_PATH, f"pointCloud{id}.ply"))
    param_file = open(os.path.join(SAVE_PARAM_PATH, f"pointCloud{id}_prediction.txt"), "w") 

    try:
        generate_shape_params(pc, param_file, name[label])
    except:
        print(f"[+] Failed at id: {id}, shape: {name[label]}")


  0%|          | 0/15 [00:00<?, ?it/s]